# Word2Vec Pyvis

In [1]:
import pandas as pd
import numpy as np
import pyvis
import random
import networkx as nx
from pyvis.network import Network

In [2]:
# 編碼解決方法  
import _locale
_locale._getdefaultlocale = (lambda *args: ['en_US', 'utf8'])
_locale._getdefaultlocale

<function __main__.<lambda>(*args)>

In [3]:
df = pd.read_csv('./rawData/nissan_correlation.csv')
df = df.drop(columns=['Unnamed: 0'], axis=1)
df

,item1,item2,correlation
0,nissan,nnissan,0.667420
1,nissan,nismo,0.574859
2,nissan,今日,0.560769
3,nissan,裕隆日產,0.556396
4,nissan,主角,0.541733
...,...,...,...
1995,台灣人,照樣,0.711645
1996,台灣人,好意思,0.709959
1997,台灣人,中共,0.707242
1998,台灣人,看不起,0.702805


In [4]:
print(df['item1'].dtype)
print(df['item2'].dtype)

object
object


In [5]:
data = df.copy()
data = data[~data.item1.isna()]
data = data[~data.item2.isna()]
data

,item1,item2,correlation
0,nissan,nnissan,0.667420
1,nissan,nismo,0.574859
2,nissan,今日,0.560769
3,nissan,裕隆日產,0.556396
4,nissan,主角,0.541733
...,...,...,...
1995,台灣人,照樣,0.711645
1996,台灣人,好意思,0.709959
1997,台灣人,中共,0.707242
1998,台灣人,看不起,0.702805


In [6]:
data = data[(data['correlation'] <= 0.75) & (data['correlation'] > 0.7)]
data

,item1,item2,correlation
80,業務,業代,0.745061
90,空間,寬敞,0.704260
120,油電,hybrid,0.702189
240,focus,active,0.729345
401,後座,膝部,0.739037
...,...,...,...
1995,台灣人,照樣,0.711645
1996,台灣人,好意思,0.709959
1997,台灣人,中共,0.707242
1998,台灣人,看不起,0.702805


計算 adjency matrix

In [7]:
pers = np.unique(data[['item1', 'item2']])
mat = pd.pivot_table(data,index = 'item1', columns = 'item2' ,values='correlation').fillna(0)\
  .reindex(columns=pers, index=pers, fill_value=0).to_numpy() # 有向圖
print(mat.shape)
tri = (np.tril(mat,-1).T + np.triu(mat,1))
mat_s = tri+tri.T

(291, 291)


繪製網路圖

In [18]:
def random_color():
  r = lambda: random.randint(0,255)
  return '#%02X%02X%02X' % (r(),r(),r())

# input matrix, output graph
def matPresentGraph(mat:np.array,node_id:list,node_value:list = None,edge_color=None):

  if edge_color == None:
    edge_color = random_color()
  edge_color = random_color()
  if node_value == None:
    node_value = [1 for i in range(len(node_id))]

  # cdn_resource : 指定網路圖的CDN資源  
  net = Network(notebook=True, cdn_resources='in_line')

  titles_list = []
  for i in zip(node_id):
    titles_list.append(str(i))

  net.add_nodes(
      nodes = node_id,
      value = node_value,
      label = node_id,
      title = titles_list
  )

  for row in range(len(node_id)):
    for col in range(len(node_id)):
      if mat[row][col]>0.:
        net.add_edge(
            node_id[row],node_id[col],width = mat[row][col],color = edge_color,title = mat[row][col]
        )

  # 計算layout，產生更易讀的網路圖 (使節點散開、避免節點重疊)
  net.repulsion()
  return net


In [20]:
net = matPresentGraph(mat = mat,node_id = pers)

net.show("./nissan_w2v.html")

291
291
291
./nissan_w2v.html


# Data Setting

In [48]:
metaData = pd.read_csv('./output/nissan_ptt_clean.csv')
metaData

,Unnamed: 0,system_id,artTitle,artDate,artPoster,artCatagory,word
0,0,1,[新聞]小休旅熱鬧好玩PEUGEOT30081.5LBlueH,2020/12/1 00:09,city0504,car,"['Corolla', 'Cross', '當紅', '炸子雞', 'NISSAN', 'K..."
1,0,1,[新聞]小休旅熱鬧好玩PEUGEOT30081.5LBlueH,2020/12/1 00:09,city0504,car,"['一網打盡', '不同', '消費', '需求', 'VW', 'TRocT', 'Cro..."
2,0,1,[新聞]小休旅熱鬧好玩PEUGEOT30081.5LBlueH,2020/12/1 00:09,city0504,car,"['尺碼', '規配', '迥異', '獨立', '車型', '吸引', '不同', '個性..."
3,0,1,[新聞]小休旅熱鬧好玩PEUGEOT30081.5LBlueH,2020/12/1 00:09,city0504,car,"['PEUGEOT', '3008', '單是', '引進', '國內', '車款', '選擇']"
4,0,1,[新聞]小休旅熱鬧好玩PEUGEOT30081.5LBlueH,2020/12/1 00:09,city0504,car,"['引擎動力', '汽油', '柴油']"
...,...,...,...,...,...,...,...
71827,2324,2325,[售車]自售NISSANTEANAJ3120052.0,2023/1/26 17:42,efs92e01,CarShop,"['電話', '0919911030LINE', '電話', '號碼', '聯繫']"
71828,2324,2325,[售車]自售NISSANTEANAJ3120052.0,2023/1/26 17:42,efs92e01,CarShop,"['不要', '半夜', '打給']"
71829,2325,2326,售Kicks原廠避光墊,2023/1/30 21:44,tingiy,CarShop,"['物品', '名稱', 'Nissan', 'kicks', '避光', '物品', '狀..."
71830,2325,2326,售Kicks原廠避光墊,2023/1/30 21:44,tingiy,CarShop,"['中壢', '車站', '輻射', '向外', '15', '公里', '宅配', '欲售..."


In [50]:
import seaborn as sns
from gensim.models.phrases import Phrases

Bigram


In [51]:
sents = metaData['word'].to_list()
bigrams = Phrases(sents, min_count=1, threshold=1000)
bigram_phrasers = Phrases(bigrams)
metaData['word_list_bigrams'] = list(bigram_phrasers[sents])

metaData.head()

TypeError: 'int' object is not iterable